**Feature Extraction**

In [ ]:
# import os, pickle, torch, torch.nn as nn
# from torchvision import models, transforms
# from torch.utils.data import DataLoader, Dataset
# from PIL import Image
# from tqdm import tqdm

# def find_image_dir():
#     # Common Kaggle root
#     base_input = '/kaggle/input/datasets/adityajn105/flickr30k'
#     # Walk through the input directory to find where the images actually are
#     for root, dirs, files in os.walk(base_input):
#         # Look for the folder containing a high volume of jpg files
#         if len([f for f in files if f.endswith('.jpg')]) > 1000:
#             return root
#     return None

# IMAGE_DIR = find_image_dir()
# OUTPUT_FILE = 'flickr30k_features.pkl'

# if IMAGE_DIR:
#     print(f"✓ Found images at: {IMAGE_DIR}")
# else:
#     raise FileNotFoundError("Could not find the Flickr30k image directory. Please ensure the dataset is added to the notebook.")

# # --- THE DATASET CLASS ---
# class FlickrDataset(Dataset):
#     def __init__(self, img_dir, transform):
#         self.img_names = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.jpeg'))]
#         self.transform = transform
#         self.img_dir = img_dir
    
#     def __len__(self):
#         return len(self.img_names)
    
#     def __getitem__(self, idx):
#         name = self.img_names[idx]
#         img_path = os.path.join(self.img_dir, name)
#         img = Image.open(img_path).convert('RGB')
#         return self.transform(img), name

# # --- REMAINDER OF THE PIPELINE (AS BEFORE) ---
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
# model = nn.Sequential(*list(model.children())[:-1])  # Feature vector only
# model = nn.DataParallel(model).to(device)
# model.eval()

# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
# ])

# dataset = FlickrDataset(IMAGE_DIR, transform)
# loader = DataLoader(dataset, batch_size=128, num_workers=4)

# features_dict = {}
# with torch.no_grad():
#     for imgs, names in tqdm(loader, desc="Extracting Features"):
#         feats = model(imgs.to(device))
#         feats = feats.squeeze(-1).squeeze(-1)
#         for i, name in enumerate(names):
#             features_dict[name] = feats[i].cpu().numpy().astype('float32')

# with open(OUTPUT_FILE, 'wb') as f:
#     pickle.dump(features_dict, f)

# print(f"Success! {len(features_dict)} images processed and saved to {OUTPUT_FILE}")

**Normalization and Tokenization**

In [ ]:
# import csv
# import pickle
# import re
# from collections import Counter

# # --- LOAD CAPTIONS ---
# CAPTIONS_FILE = '/kaggle/input/datasets/adityajn105/flickr30k/captions.txt'

# def load_captions(file_path):
#     """Load captions from file and organize by image"""
#     captions_dict = {}
    
#     with open(file_path, 'r', encoding='utf-8', newline='') as f:
#         reader = csv.reader(f)
#         next(reader, None)  # Skip header line
#         for row in reader:
#             if not row or len(row) < 2:
#                 continue
            
#             img_caption = row[0].strip()
#             caption = ','.join(row[1:]).strip()  # Preserve commas in captions
#             if not img_caption or not caption:
#                 continue
            
#             # Extract image name (remove caption number like #0, #1, etc.)
#             img_name = img_caption.rsplit('#', 1)[0]
            
#             if img_name not in captions_dict:
#                 captions_dict[img_name] = []
#             captions_dict[img_name].append(caption)
    
#     return captions_dict

# # --- NORMALIZATION ---
# def normalize_caption(caption):
#     """Normalize caption text"""
#     if not caption:
#         return ''
    
#     # Convert to lowercase
#     caption = caption.lower()
    
#     # Remove special characters and extra spaces
#     caption = re.sub(r'[^a-z0-9\s]', '', caption)
    
#     # Remove multiple spaces
#     caption = re.sub(r'\s+', ' ', caption)
    
#     return caption.strip()

# # --- TOKENIZATION ---
# def tokenize_caption(caption):
#     """Word-level tokenization"""
#     return caption.split()

# # --- BUILD VOCABULARY ---
# def build_vocabulary(captions_dict, min_freq=5):
#     """Build vocabulary from all captions"""
#     word_counter = Counter()
    
#     # Count all words
#     for img_name, captions in captions_dict.items():
#         for caption in captions:
#             normalized = normalize_caption(caption)
#             if not normalized:
#                 continue
#             tokens = tokenize_caption(normalized)
#             word_counter.update(tokens)
    
#     # Filter by minimum frequency
#     vocab = [word for word, freq in word_counter.items() if freq >= min_freq]
    
#     # Sort vocabulary for consistency
#     vocab = sorted(vocab)
    
#     return vocab, word_counter

# # --- CREATE TOKEN MAPPINGS ---
# def create_token_mappings(vocab):
#     """Create word-to-index and index-to-word mappings"""
#     # Special tokens
#     SPECIAL_TOKENS = {
#         '<PAD>': 0,
#         '<START>': 1,
#         '<END>': 2,
#         '<UNK>': 3  # Unknown token for words not in vocabulary
#     }
    
#     # Start regular vocabulary after special tokens
#     word2idx = SPECIAL_TOKENS.copy()
#     idx2word = {idx: word for word, idx in SPECIAL_TOKENS.items()}
    
#     # Add vocabulary words
#     for idx, word in enumerate(vocab, start=len(SPECIAL_TOKENS)):
#         word2idx[word] = idx
#         idx2word[idx] = word
    
#     return word2idx, idx2word

# # --- ENCODE CAPTIONS ---
# def encode_caption(caption, word2idx, add_special_tokens=True):
#     """Convert caption to token IDs"""
#     normalized = normalize_caption(caption)
#     tokens = tokenize_caption(normalized) if normalized else []
    
#     # Convert words to indices
#     token_ids = []
    
#     if add_special_tokens:
#         token_ids.append(word2idx['<START>'])
    
#     for token in tokens:
#         # Use <UNK> token for words not in vocabulary
#         token_ids.append(word2idx.get(token, word2idx['<UNK>']))
    
#     if add_special_tokens:
#         token_ids.append(word2idx['<END>'])
    
#     return token_ids

# # --- MAIN PROCESSING ---
# print("Loading captions...")
# captions_dict = load_captions(CAPTIONS_FILE)
# print(f"Loaded captions for {len(captions_dict)} images")

# print("\nBuilding vocabulary...")
# vocab, word_counter = build_vocabulary(captions_dict, min_freq=5)
# print(f"Vocabulary size: {len(vocab)} words (min frequency: 5)")
# print(f"Total unique words before filtering: {len(word_counter)}")

# print("\nCreating token mappings...")
# word2idx, idx2word = create_token_mappings(vocab)
# print(f"Total vocabulary size (including special tokens): {len(word2idx)}")

# # --- ENCODE ALL CAPTIONS ---
# print("\nEncoding all captions...")
# encoded_captions = {}
# for img_name, captions in captions_dict.items():
#     encoded_captions[img_name] = []
#     for caption in captions:
#         token_ids = encode_caption(caption, word2idx, add_special_tokens=True)
#         encoded_captions[img_name].append(token_ids)

# # --- SAVE EVERYTHING ---
# output_data = {
#     'captions_dict': captions_dict,  # Original captions
#     'encoded_captions': encoded_captions,  # Tokenized captions
#     'word2idx': word2idx,  # Word to index mapping
#     'idx2word': idx2word,  # Index to word mapping
#     'vocab': vocab,  # Vocabulary list
#     'word_counter': word_counter  # Word frequencies
# }

# OUTPUT_FILE = 'flickr30k_captions_processed.pkl'
# with open(OUTPUT_FILE, 'wb') as f:
#     pickle.dump(output_data, f)

# print(f"\n✓ Success! Saved to {OUTPUT_FILE}")

# # --- PRINT STATISTICS ---
# print("\n" + "="*60)
# print("STATISTICS")
# print("="*60)
# print(f"Special Tokens:")
# print(f"  <PAD>: {word2idx['<PAD>']}")
# print(f"  <START>: {word2idx['<START>']}")
# print(f"  <END>: {word2idx['<END>']}")
# print(f"  <UNK>: {word2idx['<UNK>']}")
# print(f"\nVocabulary size: {len(word2idx)}")
# print(f"Number of images: {len(captions_dict)}")
# print(f"Total captions: {sum(len(caps) for caps in captions_dict.values())}")

# # Calculate average caption length
# all_lengths = []
# for caps in encoded_captions.values():
#     for cap in caps:
#         all_lengths.append(len(cap))
# print(f"Average caption length (with special tokens): {sum(all_lengths) / len(all_lengths):.2f}")

# # --- SHOW EXAMPLE ---
# print("\n" + "="*60)
# print("EXAMPLE")
# print("="*60)
# sample_img = list(captions_dict.keys())[0]
# print(f"Image: {sample_img}")
# print(f"\nOriginal caption:")
# print(f"  {captions_dict[sample_img][0]}")
# print(f"\nNormalized caption:")
# normalized = normalize_caption(captions_dict[sample_img][0])
# print(f"  {normalized}")
# print(f"\nTokenized:")
# print(f"  {tokenize_caption(normalized)}")
# print(f"\nEncoded (token IDs):")
# print(f"  {encoded_captions[sample_img][0]}")
# print(f"\nDecoded back:")
# decoded = [idx2word[idx] for idx in encoded_captions[sample_img][0]]
# print(f"  {decoded}")

**Training Code LSTM**

In [ ]:
# import pickle
# import random
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# from torch.nn.utils.rnn import pad_sequence
# import matplotlib.pyplot as plt
# from tqdm import tqdm

# # Set random seeds for reproducibility
# torch.manual_seed(42)
# np.random.seed(42)
# random.seed(42)

# # ============================================================================
# # CONFIGURATION
# # ============================================================================
# class Config:
#     # Paths
#     FEATURES_PATH = '/kaggle/input/genai-a1-22f3244-22f8758/archive (1)/flickr30k_features.pkl'
#     CAPTIONS_PATH = '/kaggle/input/genai-a1-22f3244-22f8758/archive (1)/flickr30k_captions_processed.pkl'
    
#     # Model Architecture
#     IMAGE_FEATURE_DIM = 2048
#     HIDDEN_DIM = 512
#     ENCODER_LAYERS = 2
#     DECODER_LAYERS = 2
#     EMBEDDING_DIM = 512
#     DROPOUT = 0.5
    
#     # Training
#     BATCH_SIZE = 64
#     LEARNING_RATE = 0.0005
#     NUM_EPOCHS = 30
#     TEACHER_FORCING_RATIO_START = 0.9
#     TEACHER_FORCING_RATIO_END = 0.5
#     MAX_CAPTION_LENGTH = 50
#     LABEL_SMOOTHING = 0.1
    
#     # Learning Rate Scheduling
#     LR_PATIENCE = 2
#     LR_FACTOR = 0.5
    
#     # Beam Search
#     BEAM_WIDTH = 5
    
#     # Data Split
#     TRAIN_SPLIT = 0.8
#     VAL_SPLIT = 0.1
    
#     # Device
#     DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# config = Config()

# # ============================================================================
# # LOAD DATA
# # ============================================================================
# print("Loading data...")
# with open(config.FEATURES_PATH, 'rb') as f:
#     features_dict = pickle.load(f)

# with open(config.CAPTIONS_PATH, 'rb') as f:
#     captions_data = pickle.load(f)

# encoded_captions = captions_data['encoded_captions']
# word2idx = captions_data['word2idx']
# idx2word = captions_data['idx2word']
# vocab_size = len(word2idx)
# pad_idx = word2idx['<PAD>']

# common_images = set(features_dict.keys()) & set(encoded_captions.keys())
# if not common_images:
#     raise ValueError("No overlap between image features and captions. Check filenames and paths.")
# sample_feature = next(iter(features_dict.values()))
# if sample_feature.shape[0] != config.IMAGE_FEATURE_DIM:
#     raise ValueError(
#         f"Feature dim mismatch: expected {config.IMAGE_FEATURE_DIM}, got {sample_feature.shape[0]}"
#     )
# print(f"✓ Loaded {len(features_dict)} image features")
# print(f"✓ Loaded captions for {len(encoded_captions)} images")
# print(f"✓ Overlap images: {len(common_images)}")
# print(f"✓ Vocabulary size: {vocab_size}")
# print(f"✓ Device: {config.DEVICE}")

# # ============================================================================
# # DATASET CLASS
# # ============================================================================
# class FlickrDataset(Dataset):
#     def __init__(self, image_names, features_dict, encoded_captions, max_length):
#         self.data = []
        
#         for img_name in image_names:
#             if img_name in features_dict and img_name in encoded_captions:
#                 feature = features_dict[img_name]
#                 for caption in encoded_captions[img_name]:
#                     if len(caption) <= max_length:
#                         self.data.append((feature, caption, img_name))
    
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         feature, caption, img_name = self.data[idx]
#         return torch.FloatTensor(feature), torch.LongTensor(caption), img_name

# def collate_fn(batch):
#     features, captions, img_names = zip(*batch)
#     features = torch.stack(features)
#     captions_padded = pad_sequence(captions, batch_first=True, padding_value=pad_idx)
#     return features, captions_padded, img_names

# # ============================================================================
# # SPLIT DATA
# # ============================================================================
# all_image_names = list(common_images)
# random.shuffle(all_image_names)

# n_total = len(all_image_names)
# n_train = int(n_total * config.TRAIN_SPLIT)
# n_val = int(n_total * config.VAL_SPLIT)

# train_names = all_image_names[:n_train]
# val_names = all_image_names[n_train:n_train + n_val]
# test_names = all_image_names[n_train + n_val:]

# print(f"\nData Split:")
# print(f"  Train: {len(train_names)} images")
# print(f"  Val: {len(val_names)} images")
# print(f"  Test: {len(test_names)} images")

# train_dataset = FlickrDataset(train_names, features_dict, encoded_captions, config.MAX_CAPTION_LENGTH)
# val_dataset = FlickrDataset(val_names, features_dict, encoded_captions, config.MAX_CAPTION_LENGTH)
# test_dataset = FlickrDataset(test_names, features_dict, encoded_captions, config.MAX_CAPTION_LENGTH)

# train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
# val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
# test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# print(f"  Train batches: {len(train_loader)}")
# print(f"  Val batches: {len(val_loader)}")
# print(f"  Test batches: {len(test_loader)}")

# # ============================================================================
# # LSTM ENCODER (FROM SCRATCH VIA LSTMCell)
# # ============================================================================
# class LSTMEncoder(nn.Module):
#     def __init__(self, input_dim, hidden_dim, num_layers, dropout):
#         super(LSTMEncoder, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.num_layers = num_layers
        
#         self.feature_projection = nn.Sequential(
#             nn.Linear(input_dim, hidden_dim),
#             nn.LayerNorm(hidden_dim),
#             nn.ReLU(),
#             nn.Dropout(dropout)
#         )
        
#         self.h_init = nn.Linear(hidden_dim, hidden_dim)
#         self.c_init = nn.Linear(hidden_dim, hidden_dim)
        
#         self.dropout = nn.Dropout(dropout)
    
#     def forward(self, image_features):
#         x = self.feature_projection(image_features)
#         h0 = torch.tanh(self.h_init(x))
#         c0 = torch.tanh(self.c_init(x))
        
#         h_states = []
#         c_states = []
#         for _ in range(self.num_layers):
#             h_states.append(self.dropout(h0))
#             c_states.append(c0)
        
#         h_states = torch.stack(h_states)
#         c_states = torch.stack(c_states)
#         return h_states, c_states

# # ============================================================================
# # LSTM DECODER (FROM SCRATCH VIA LSTMCell)
# # ============================================================================
# class LSTMDecoder(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout):
#         super(LSTMDecoder, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.num_layers = num_layers
#         self.vocab_size = vocab_size
        
#         self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
#         self.lstm_cells = nn.ModuleList([
#             nn.LSTMCell(embedding_dim if i == 0 else hidden_dim, hidden_dim)
#             for i in range(num_layers)
#         ])
#         self.layer_norms = nn.ModuleList([
#             nn.LayerNorm(hidden_dim) for _ in range(num_layers)
#         ])
#         self.dropout = nn.Dropout(dropout)
#         self.fc_out = nn.Linear(hidden_dim, vocab_size)
    
#     def forward(self, input_token, hidden_states, cell_states):
#         embedded = self.embedding(input_token)
#         embedded = self.dropout(embedded)
        
#         x = embedded
#         new_h_states = []
#         new_c_states = []
        
#         for layer_idx in range(self.num_layers):
#             h_prev = hidden_states[layer_idx]
#             c_prev = cell_states[layer_idx]
            
#             h, c = self.lstm_cells[layer_idx](x, (h_prev, c_prev))
#             h = self.layer_norms[layer_idx](h)
            
#             new_h_states.append(h)
#             new_c_states.append(c)
#             x = self.dropout(h)
        
#         output = self.fc_out(x)
#         new_h_states = torch.stack(new_h_states)
#         new_c_states = torch.stack(new_c_states)
#         return output, new_h_states, new_c_states

# # ============================================================================
# # SEQ2SEQ MODEL WITHOUT ATTENTION
# # ============================================================================
# class Seq2SeqImageCaptioning(nn.Module):
#     def __init__(self, encoder, decoder, device):
#         super(Seq2SeqImageCaptioning, self).__init__()
#         self.encoder = encoder
#         self.decoder = decoder
#         self.device = device
    
#     def forward(self, image_features, captions, teacher_forcing_ratio=0.5):
#         batch_size = image_features.size(0)
#         max_length = captions.size(1)
        
#         hidden_states, cell_states = self.encoder(image_features)
#         outputs = torch.zeros(batch_size, max_length, self.decoder.vocab_size).to(self.device)
#         input_token = captions[:, 0]
        
#         for t in range(1, max_length):
#             output, hidden_states, cell_states = self.decoder(
#                 input_token, hidden_states, cell_states
#             )
#             outputs[:, t, :] = output
            
#             teacher_force = random.random() < teacher_forcing_ratio
#             top1 = output.argmax(1)
#             input_token = captions[:, t] if teacher_force else top1
        
#         return outputs

# # ============================================================================
# # INITIALIZE MODEL
# # ============================================================================
# encoder = LSTMEncoder(
#     input_dim=config.IMAGE_FEATURE_DIM,
#     hidden_dim=config.HIDDEN_DIM,
#     num_layers=config.ENCODER_LAYERS,
#     dropout=config.DROPOUT
#  )

# decoder = LSTMDecoder(
#     vocab_size=vocab_size,
#     embedding_dim=config.EMBEDDING_DIM,
#     hidden_dim=config.HIDDEN_DIM,
#     num_layers=config.DECODER_LAYERS,
#     dropout=config.DROPOUT
#  )

# model = Seq2SeqImageCaptioning(encoder, decoder, config.DEVICE).to(config.DEVICE)

# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

# print(f"\n✓ Model initialized with {count_parameters(model):,} trainable parameters")

# # ============================================================================
# # LOSS AND OPTIMIZER
# # ============================================================================
# criterion = nn.CrossEntropyLoss(
#     ignore_index=pad_idx,
#     label_smoothing=config.LABEL_SMOOTHING
#  )

# optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer,
#     mode='min',
#     factor=config.LR_FACTOR,
#     patience=config.LR_PATIENCE
#  )

# # ============================================================================
# # TRAINING FUNCTIONS
# # ============================================================================
# def get_teacher_forcing_ratio(epoch, start_ratio, end_ratio, num_epochs):
#     return end_ratio + (start_ratio - end_ratio) * (1 - epoch / num_epochs)

# def compute_accuracy(logits, targets, pad_token):
#     preds = logits.argmax(dim=-1)
#     mask = targets != pad_token
#     correct = (preds == targets) & mask
#     total = mask.sum().item()
#     correct_count = correct.sum().item()
#     return correct_count, total

# def train_epoch(model, loader, optimizer, criterion, device, teacher_forcing_ratio):
#     model.train()
#     epoch_loss = 0
#     correct_total = 0
#     token_total = 0
    
#     for features, captions, _ in tqdm(loader, desc="Training", leave=False):
#         features = features.to(device)
#         captions = captions.to(device)
        
#         optimizer.zero_grad()
#         outputs = model(features, captions, teacher_forcing_ratio)
#         logits = outputs[:, 1:, :]
#         targets = captions[:, 1:]
        
#         loss = criterion(logits.reshape(-1, vocab_size), targets.reshape(-1))
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
#         optimizer.step()
#         epoch_loss += loss.item()
        
#         correct, total = compute_accuracy(logits, targets, pad_idx)
#         correct_total += correct
#         token_total += total
    
#     avg_loss = epoch_loss / len(loader)
#     avg_acc = correct_total / max(token_total, 1)
#     return avg_loss, avg_acc

# def evaluate(model, loader, criterion, device):
#     model.eval()
#     epoch_loss = 0
#     correct_total = 0
#     token_total = 0
    
#     with torch.no_grad():
#         for features, captions, _ in tqdm(loader, desc="Evaluating", leave=False):
#             features = features.to(device)
#             captions = captions.to(device)
#             outputs = model(features, captions, teacher_forcing_ratio=1.0)
#             logits = outputs[:, 1:, :]
#             targets = captions[:, 1:]
#             loss = criterion(logits.reshape(-1, vocab_size), targets.reshape(-1))
#             epoch_loss += loss.item()
            
#             correct, total = compute_accuracy(logits, targets, pad_idx)
#             correct_total += correct
#             token_total += total
    
#     avg_loss = epoch_loss / len(loader)
#     avg_acc = correct_total / max(token_total, 1)
#     return avg_loss, avg_acc

# # ============================================================================
# # TRAINING LOOP
# # ============================================================================
# print("\n" + "="*60)
# print("TRAINING")
# print("="*60)

# train_losses = []
# val_losses = []
# train_accs = []
# val_accs = []
# learning_rates = []
# best_val_loss = float('inf')

# for epoch in range(config.NUM_EPOCHS):
#     teacher_forcing_ratio = get_teacher_forcing_ratio(
#         epoch,
#         config.TEACHER_FORCING_RATIO_START,
#         config.TEACHER_FORCING_RATIO_END,
#         config.NUM_EPOCHS
#     )
    
#     print(f"\nEpoch {epoch+1}/{config.NUM_EPOCHS} (TF Ratio: {teacher_forcing_ratio:.3f})")
#     train_loss, train_acc = train_epoch(
#         model, train_loader, optimizer, criterion,
#         config.DEVICE, teacher_forcing_ratio
#     )
#     val_loss, val_acc = evaluate(model, val_loader, criterion, config.DEVICE)
    
#     train_losses.append(train_loss)
#     val_losses.append(val_loss)
#     train_accs.append(train_acc)
#     val_accs.append(val_acc)
#     learning_rates.append(optimizer.param_groups[0]['lr'])
    
#     print(f"  Train Loss: {train_loss:.4f}")
#     print(f"  Val Loss: {val_loss:.4f}")
#     print(f"  Train Acc: {train_acc:.4f}")
#     print(f"  Val Acc: {val_acc:.4f}")
#     print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    
#     scheduler.step(val_loss)
    
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         torch.save(model.state_dict(), 'best_model_improved.pth')
#         print("  ✓ Best model saved!")

# # ============================================================================
# # PLOT LOSSES AND LEARNING RATE
# # ============================================================================
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
# ax1.plot(range(1, config.NUM_EPOCHS + 1), train_losses, label='Training Loss', marker='o')
# ax1.plot(range(1, config.NUM_EPOCHS + 1), val_losses, label='Validation Loss', marker='s')
# ax1.set_xlabel('Epoch')
# ax1.set_ylabel('Loss')
# ax1.set_title('Training and Validation Loss Over Epochs')
# ax1.legend()
# ax1.grid(True)

# ax2.plot(range(1, config.NUM_EPOCHS + 1), learning_rates, label='Learning Rate', marker='o', color='green')
# ax2.set_xlabel('Epoch')
# ax2.set_ylabel('Learning Rate')
# ax2.set_title('Learning Rate Schedule')
# ax2.legend()
# ax2.grid(True)

# plt.tight_layout()
# plt.savefig('training_improved.png', dpi=300, bbox_inches='tight')
# plt.show()

# print("\n✓ Training complete!")

# # ============================================================================
# # BEAM SEARCH CAPTION GENERATION
# # ============================================================================
# def beam_search(model, image_feature, word2idx, idx2word, beam_width=5, max_length=50, device='cpu'):
#     model.eval()
    
#     with torch.no_grad():
#         image_feature = image_feature.unsqueeze(0).to(device)
#         hidden_states, cell_states = model.encoder(image_feature)
        
#         start_token = word2idx['<START>']
#         beams = [([start_token], 0.0, hidden_states, cell_states)]
#         completed_sequences = []
        
#         for _ in range(max_length):
#             candidates = []
            
#             for seq, score, h_states, c_states in beams:
#                 if seq[-1] == word2idx['<END>']:
#                     completed_sequences.append((seq, score))
#                     continue
                
#                 input_token = torch.LongTensor([seq[-1]]).to(device)
#                 output, new_h, new_c = model.decoder(input_token, h_states, c_states)
                
#                 log_probs = F.log_softmax(output, dim=-1)
#                 top_log_probs, top_indices = log_probs.topk(beam_width, dim=-1)
                
#                 for i in range(beam_width):
#                     token = top_indices[0, i].item()
#                     token_score = top_log_probs[0, i].item()
#                     new_seq = seq + [token]
#                     new_score = score + token_score
#                     candidates.append((new_seq, new_score, new_h, new_c))
            
#             if not candidates:
#                 break
            
#             candidates.sort(key=lambda x: x[1], reverse=True)
#             beams = candidates[:beam_width]
            
#             if all(seq[-1] == word2idx['<END>'] for seq, _, _, _ in beams):
#                 break
        
#         for seq, score, _, _ in beams:
#             if seq[-1] != word2idx['<END>']:
#                 completed_sequences.append((seq, score))
        
#         if not completed_sequences:
#             completed_sequences = [(seq, score) for seq, score, _, _ in beams]
        
#         best_seq, _ = max(completed_sequences, key=lambda x: x[1] / len(x[0]))
#         caption = []
#         for token_id in best_seq[1:]:
#             if token_id == word2idx['<END>']:
#                 break
#             if token_id != word2idx['<PAD>']:
#                 caption.append(idx2word[token_id])
        
#         return ' '.join(caption)

# # ============================================================================
# # EVALUATION METRICS
# # ============================================================================
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# def calculate_bleu(references, hypothesis):
#     smoothing = SmoothingFunction().method1
#     return sentence_bleu(
#         references,
#         hypothesis,
#         weights=(0.25, 0.25, 0.25, 0.25),
#         smoothing_function=smoothing
#     )

# def calculate_token_metrics(references, hypothesis):
#     ref_tokens = set()
#     for ref in references:
#         ref_tokens.update(ref)
    
#     hyp_tokens = set(hypothesis)
#     tp = len(ref_tokens & hyp_tokens)
#     fp = len(hyp_tokens - ref_tokens)
#     fn = len(ref_tokens - hyp_tokens)
    
#     precision = tp / (tp + fp) if (tp + fp) > 0 else 0
#     recall = tp / (tp + fn) if (tp + fn) > 0 else 0
#     f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
#     return precision, recall, f1

# # ============================================================================
# # EVALUATION ON TEST SET
# # ============================================================================
# print("\n" + "="*60)
# print("EVALUATION ON TEST SET")
# print("="*60)
# model.load_state_dict(torch.load('best_model_improved.pth'))
# model.eval()

# bleu_scores_beam = []
# precisions = []
# recalls = []
# f1_scores = []
# sample_predictions = []

# print("\nGenerating captions with Beam Search (this may take a while)...")

# with torch.no_grad():
#     for features, captions, img_names in tqdm(test_loader, desc="Testing"):
#         features = features.to(config.DEVICE)
        
#         for i in range(features.size(0)):
#             img_name = img_names[i]
#             generated_beam = beam_search(
#                 model, features[i].cpu(), word2idx, idx2word,
#                 beam_width=config.BEAM_WIDTH,
#                 max_length=config.MAX_CAPTION_LENGTH, device=config.DEVICE
#             )
            
#             reference_captions = []
#             for caption_ids in encoded_captions[img_name]:
#                 tokens = [idx2word[idx] for idx in caption_ids
#                          if idx not in [word2idx['<START>'], word2idx['<END>'], word2idx['<PAD>']]]
#                 reference_captions.append(tokens)
            
#             generated_tokens_beam = generated_beam.split()
#             bleu_beam = calculate_bleu(reference_captions, generated_tokens_beam)
#             bleu_scores_beam.append(bleu_beam)
            
#             precision, recall, f1 = calculate_token_metrics(reference_captions, generated_tokens_beam)
#             precisions.append(precision)
#             recalls.append(recall)
#             f1_scores.append(f1)
            
#             if len(sample_predictions) < 10:
#                 sample_predictions.append({
#                     'image': img_name,
#                     'generated_beam': generated_beam,
#                     'references': [' '.join(ref) for ref in reference_captions]
#                 })

# print(f"\n{'='*60}")
# print("QUANTITATIVE EVALUATION RESULTS")
# print(f"{'='*60}")
# print(f"\nBeam Search (width={config.BEAM_WIDTH}):")
# print(f"  BLEU-4 Score: {np.mean(bleu_scores_beam):.4f} (±{np.std(bleu_scores_beam):.4f})")
# print(f"  Precision: {np.mean(precisions):.4f} (±{np.std(precisions):.4f})")
# print(f"  Recall: {np.mean(recalls):.4f} (±{np.std(recalls):.4f})")
# print(f"  F1-Score: {np.mean(f1_scores):.4f} (±{np.std(f1_scores):.4f})")

# print("\n" + "="*60)
# print("SAMPLE PREDICTIONS")
# print("="*60)
# for i, sample in enumerate(sample_predictions[:5], 1):
#     print(f"\n{i}. Image: {sample['image']}")
#     print(f"   Generated (Beam):   {sample['generated_beam']}")
#     print(f"   References:")
#     for j, ref in enumerate(sample['references'], 1):
#         print(f"     {j}. {ref}")

# print("\n✓ Evaluation complete!")

# results = {
#     'train_losses': train_losses,
#     'val_losses': val_losses,
#     'train_accs': train_accs,
#     'val_accs': val_accs,
#     'learning_rates': learning_rates,
#     'bleu_scores_beam': bleu_scores_beam,
#     'precisions': precisions,
#     'recalls': recalls,
#     'f1_scores': f1_scores,
#     'sample_predictions': sample_predictions,
#     'config': {
#         'hidden_dim': config.HIDDEN_DIM,
#         'encoder_layers': config.ENCODER_LAYERS,
#         'decoder_layers': config.DECODER_LAYERS,
#         'beam_width': config.BEAM_WIDTH,
#         'label_smoothing': config.LABEL_SMOOTHING
#     }
# }

# with open('evaluation_results_improved.pkl', 'wb') as f:
#     pickle.dump(results, f)

# print("✓ Results saved to evaluation_results_improved.pkl")

**Inference and Gradio UI**

In [1]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import gradio as gr

# =========================
# CONFIGURATION
# =========================
class Config:
    FEATURES_PATH = "/kaggle/input/genai-a1-22f3244-22f8758/archive (1)/flickr30k_features.pkl"
    CAPTIONS_PATH = "/kaggle/input/genai-a1-22f3244-22f8758/archive (1)/flickr30k_captions_processed.pkl"
    MODEL_WEIGHTS = "/kaggle/input/genai-a1-22f3244-22f8758/archive/best_model.pth"

    IMAGE_FEATURE_DIM = 2048
    HIDDEN_DIM = 512
    ENCODER_LAYERS = 2
    DECODER_LAYERS = 2
    EMBEDDING_DIM = 512
    DROPOUT = 0.5

    BEAM_WIDTH = 5
    MAX_CAPTION_LENGTH = 50

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = Config()

# =========================
# LOAD TOKEN DATA
# =========================
with open(config.CAPTIONS_PATH, "rb") as f:
    captions_data = pickle.load(f)

word2idx = captions_data["word2idx"]
idx2word = captions_data["idx2word"]
vocab_size = len(word2idx)
pad_idx = word2idx["<PAD>"]

features_dict = {}

# =========================
# MODEL DEFINITION
# =========================
class LSTMEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout):
        super(LSTMEncoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.feature_projection = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        self.h_init = nn.Linear(hidden_dim, hidden_dim)
        self.c_init = nn.Linear(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, image_features):
        x = self.feature_projection(image_features)
        h0 = torch.tanh(self.h_init(x))
        c0 = torch.tanh(self.c_init(x))

        h_states = []
        c_states = []
        for _ in range(self.num_layers):
            h_states.append(self.dropout(h0))
            c_states.append(c0)

        h_states = torch.stack(h_states)
        c_states = torch.stack(c_states)
        return h_states, c_states

class LSTMDecoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout):
        super(LSTMDecoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.lstm_cells = nn.ModuleList([
            nn.LSTMCell(embedding_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(num_layers)
        ])
        self.layer_norms = nn.ModuleList([
            nn.LayerNorm(hidden_dim) for _ in range(num_layers)
        ])
        self.dropout = nn.Dropout(dropout)
        self.fc_out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_token, hidden_states, cell_states):
        embedded = self.embedding(input_token)
        embedded = self.dropout(embedded)

        x = embedded
        new_h_states = []
        new_c_states = []

        for layer_idx in range(self.num_layers):
            h_prev = hidden_states[layer_idx]
            c_prev = cell_states[layer_idx]

            h, c = self.lstm_cells[layer_idx](x, (h_prev, c_prev))
            h = self.layer_norms[layer_idx](h)

            new_h_states.append(h)
            new_c_states.append(c)
            x = self.dropout(h)

        output = self.fc_out(x)
        new_h_states = torch.stack(new_h_states)
        new_c_states = torch.stack(new_c_states)
        return output, new_h_states, new_c_states

class Seq2SeqImageCaptioning(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2SeqImageCaptioning, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

# =========================
# LOAD MODEL
# =========================
encoder = LSTMEncoder(
    input_dim=config.IMAGE_FEATURE_DIM,
    hidden_dim=config.HIDDEN_DIM,
    num_layers=config.ENCODER_LAYERS,
    dropout=config.DROPOUT
)

decoder = LSTMDecoder(
    vocab_size=vocab_size,
    embedding_dim=config.EMBEDDING_DIM,
    hidden_dim=config.HIDDEN_DIM,
    num_layers=config.DECODER_LAYERS,
    dropout=config.DROPOUT
)

model = Seq2SeqImageCaptioning(encoder, decoder, config.DEVICE).to(config.DEVICE)
state_dict = torch.load(config.MODEL_WEIGHTS, map_location=config.DEVICE)
model.load_state_dict(state_dict)
model.eval()

# =========================
# CNN FEATURE EXTRACTOR
# =========================
cnn = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
cnn = nn.Sequential(*list(cnn.children())[:-1]).to(config.DEVICE)
cnn.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# =========================
# BEAM SEARCH
# =========================
def beam_search(image_feature, beam_width=5, max_length=50):
    with torch.no_grad():
        image_feature = image_feature.unsqueeze(0).to(config.DEVICE)
        hidden_states, cell_states = model.encoder(image_feature)

        start_token = word2idx["<START>"]
        beams = [([start_token], 0.0, hidden_states, cell_states)]
        completed_sequences = []

        for _ in range(max_length):
            candidates = []

            for seq, score, h_states, c_states in beams:
                if seq[-1] == word2idx["<END>"]:
                    completed_sequences.append((seq, score))
                    continue

                input_token = torch.LongTensor([seq[-1]]).to(config.DEVICE)
                output, new_h, new_c = model.decoder(input_token, h_states, c_states)

                log_probs = F.log_softmax(output, dim=-1)
                top_log_probs, top_indices = log_probs.topk(beam_width, dim=-1)

                for i in range(beam_width):
                    token = top_indices[0, i].item()
                    token_score = top_log_probs[0, i].item()
                    new_seq = seq + [token]
                    new_score = score + token_score
                    candidates.append((new_seq, new_score, new_h, new_c))

            if not candidates:
                break

            candidates.sort(key=lambda x: x[1], reverse=True)
            beams = candidates[:beam_width]

            if all(seq[-1] == word2idx["<END>"] for seq, _, _, _ in beams):
                break

        for seq, score, _, _ in beams:
            if seq[-1] != word2idx["<END>"]:
                completed_sequences.append((seq, score))

        if not completed_sequences:
            completed_sequences = [(seq, score) for seq, score, _, _ in beams]

        best_seq, _ = max(completed_sequences, key=lambda x: x[1] / len(x[0]))

        caption = []
        for token_id in best_seq[1:]:
            if token_id == word2idx["<END>"]:
                break
            if token_id != word2idx["<PAD>"]:
                caption.append(idx2word[token_id])

        return " ".join(caption)

# =========================
# INFERENCE
# =========================
def extract_feature_from_image(pil_image):
    img_tensor = transform(pil_image).unsqueeze(0).to(config.DEVICE)
    with torch.no_grad():
        feats = cnn(img_tensor)
        feats = feats.squeeze(-1).squeeze(-1)
    return feats[0].cpu()

def generate_caption(image):
    if image is None:
        return "No image provided."

    feature = extract_feature_from_image(image)

    caption = beam_search(feature, beam_width=config.BEAM_WIDTH, max_length=config.MAX_CAPTION_LENGTH)
    return caption

# =========================
# GRADIO UI
# =========================
with gr.Blocks() as demo:
    gr.Markdown("# Flickr30k Image Captioning")
    gr.Markdown("Upload any image to generate a caption.")
    image_input = gr.Image(type="pil", label="Input Image")
    output_text = gr.Textbox(label="Generated Caption")
    submit_btn = gr.Button("Generate")

    def _predict(image):
        return generate_caption(image)

    submit_btn.click(
        _predict,
        inputs=[image_input],
        outputs=output_text
    )

if __name__ == "__main__":
    demo.launch()


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 207MB/s] 


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://74a8359d23825e519a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
